In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, recall_score, precision_score ,f1_score
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix

from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neural_network import MLPClassifier

import pickle
from joblib import dump, load

import cv2
import mediapipe as mp

import os

In [2]:
 

def label_landmarks(landmarks, label):
    unboxed_landmarks = {}
    unboxed_landmarks['label'] = label 
    
    
    for i in range(33):
        unboxed_landmarks['x' + str(i)] = landmarks.landmark[i].x
        unboxed_landmarks['y' + str(i)] = landmarks.landmark[i].y
        unboxed_landmarks['z' + str(i)] = landmarks.landmark[i].z
        
    all_landmarks.append(unboxed_landmarks)

In [3]:
pt="/0.Jupiter notebook/major-1"

# --------------------------------------------------Squat----------------------------------------------------

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
exit = False
folder_name="squat"
path=f"{pt}/{folder_name}"


all_videos = []

for file in os.listdir(path):
    
    all_videos.append(os.fsdecode(file))

In [5]:
all_landmarks = []
for filename in all_videos:
    print(filename)
    cap = cv2.VideoCapture(folder_name + '/' + filename)
    with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.5) as pose:

        while cap.isOpened():
            ret, image = cap.read()

            if ret == False:
                break
            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            cv2.rectangle(image, (0,0), (225, 100), (0,0,0), -1)

            k = cv2.waitKey(1)
            
            if results.pose_landmarks is not None:      
                
                if k == 112: #ASCII of p
                    label_landmarks(results.pose_landmarks, 'parallel')
                    cv2.putText(image, 'parallel', (60,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

                
                if k == 108:  #ASCII of l
                    label_landmarks(results.pose_landmarks, 'leaning')
                    cv2.putText(image, 'leaning', (60,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

                
                if k == 103:  #ASCII of g
                    label_landmarks(results.pose_landmarks, 'good')
                    cv2.putText(image, 'good', (60,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            # Press ESC to quit
            if k == 27:
                exit = True
                break

            cv2.namedWindow('output', cv2.WINDOW_NORMAL) # Allows you to manually change the size of the window
            cv2.imshow('output', image)

        cap.release()
        cv2.destroyAllWindows()
        if exit == True:
            break

back.mp4


In [6]:
landmarks_df = pd.DataFrame(all_landmarks)
#landmarks_df
landmarks_df.to_csv("squat.csv", index=False)

,label,x0,y0,z0,x1,y1,z1,x2,y2,z2,...,z29,x30,y30,z30,x31,y31,z31,x32,y32,z32
0,good,0.504303,0.383179,0.558457,0.495864,0.373131,0.496114,0.490455,0.371738,0.496095,...,0.636678,0.497464,0.845558,-0.039796,0.392486,0.789130,0.675631,0.609184,0.824472,-0.023158
1,good,0.503573,0.396776,0.548659,0.495629,0.386672,0.487695,0.489944,0.385208,0.487671,...,0.749614,0.496623,0.844844,-0.007421,0.386487,0.788690,0.803936,0.609509,0.824875,0.003485
2,good,0.522469,0.424071,0.539298,0.515146,0.413481,0.487354,0.509884,0.412093,0.487312,...,0.723442,0.496930,0.840812,0.052308,0.388958,0.788870,0.782978,0.609762,0.823834,0.101975
3,good,0.519592,0.438355,0.544210,0.512446,0.427627,0.492726,0.506794,0.426132,0.492684,...,0.723255,0.503726,0.841464,0.059066,0.395958,0.789153,0.781738,0.609502,0.823604,0.115647
4,good,0.517372,0.441682,0.550400,0.511018,0.431912,0.497495,0.504842,0.430537,0.497463,...,0.851275,0.503473,0.840783,0.116200,0.394823,0.789216,0.919850,0.609259,0.823682,0.169680
5,good,0.530818,0.461121,0.630380,0.525758,0.451003,0.590985,0.521541,0.449828,0.590960,...,0.532121,0.502977,0.839789,-0.029264,0.396444,0.790945,0.593439,0.609567,0.824255,0.041948


In [7]:
landmarks_df = pd.read_csv('squat-main.csv')
landmarks_df

,label,x0,y0,z0,x1,y1,z1,x2,y2,z2,...,z29,x30,y30,z30,x31,y31,z31,x32,y32,z32
0,good,0.290136,0.350631,0.317024,0.279842,0.338213,0.243598,0.273157,0.337131,0.243568,...,0.209979,0.294017,0.979269,0.023859,0.171485,0.932095,0.151369,0.322534,0.946116,-0.070519
1,good,0.290249,0.350588,0.295852,0.279395,0.338053,0.218723,0.272434,0.337068,0.218684,...,0.322945,0.293214,0.979978,0.157935,0.177162,0.931930,0.260085,0.322846,0.945750,0.054738
2,good,0.298653,0.354434,0.170526,0.287318,0.340932,0.120378,0.281985,0.340247,0.120339,...,0.398127,0.292129,0.974259,0.131568,0.201440,0.925654,0.323522,0.324432,0.943259,0.020762
3,good,0.308099,0.356125,0.249306,0.295208,0.342627,0.186238,0.287099,0.341797,0.186211,...,0.294459,0.286640,0.973877,0.118518,0.170081,0.926647,0.216061,0.304217,0.941574,0.008979
4,good,0.307347,0.355517,0.282541,0.294948,0.342408,0.225469,0.286710,0.341644,0.225506,...,0.312024,0.287773,0.973791,0.111186,0.178354,0.926599,0.228849,0.309655,0.941193,0.001115
5,good,0.305400,0.356065,0.341869,0.291655,0.342504,0.296744,0.284963,0.341728,0.296814,...,0.330217,0.289048,0.972151,0.053601,0.191883,0.927479,0.229136,0.328856,0.941115,-0.083117
6,good,0.301901,0.356357,0.368135,0.286460,0.342825,0.329810,0.279594,0.342023,0.329901,...,0.275585,0.291001,0.970641,-0.015991,0.191931,0.927572,0.187712,0.332448,0.940690,-0.128182
7,good,0.293912,0.359575,0.292541,0.279189,0.346293,0.269965,0.272120,0.345313,0.270015,...,0.240053,0.294098,0.967919,0.056010,0.192514,0.932216,0.188468,0.338940,0.941130,-0.016227
8,good,0.285830,0.359969,0.272771,0.272884,0.346871,0.240164,0.265883,0.346001,0.240213,...,0.239603,0.279572,0.968609,0.051583,0.196072,0.929643,0.183604,0.335482,0.942311,-0.008521
9,good,0.287682,0.359736,0.206390,0.274377,0.346019,0.179987,0.268011,0.345229,0.180008,...,0.234610,0.282641,0.967751,0.052279,0.199902,0.931259,0.179133,0.333002,0.942171,-0.011476


# -----------------------------------------Overhead Press-----------------------------------------------

In [24]:

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
exit = False
folder_name="overhead"
path=f"{pt}/{folder_name}"


all_videos = []

for file in os.listdir(path):
    
    all_videos.append(os.fsdecode(file))

In [27]:
all_landmarks = []
for filename in all_videos:
    print(filename)
    cap = cv2.VideoCapture(folder_name + '/' + filename)
    with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.5) as pose:

        while cap.isOpened():
            ret, image = cap.read()

            if ret == False:
                break
            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            cv2.rectangle(image, (0,0), (225, 100), (0,0,0), -1)

            k = cv2.waitKey(1)
            
            if results.pose_landmarks is not None:      
                
                if k == 103:  #ASCII of g
                    label_landmarks(results.pose_landmarks, 'good')
                    cv2.putText(image, 'good', (60,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
               
                if k == 98:  #ASCII of b
                    label_landmarks(results.pose_landmarks, 'good')
                    cv2.putText(image, 'back', (60,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
          
                if k == 102:  #ASCII of f
                    label_landmarks(results.pose_landmarks, 'forward')
                    cv2.putText(image, 'forward', (60,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
          
                if k == 107:  #ASCII of k
                    label_landmarks(results.pose_landmarks, 'knees')
                    cv2.putText(image, 'knees', (60,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
          
                
            # Press ESC to quit
            if k == 27:
                exit = True
                break

            cv2.namedWindow('output', cv2.WINDOW_NORMAL) # Allows you to manually change the size of the window
            cv2.imshow('output', image)

        cap.release()
        cv2.destroyAllWindows()
        if exit == True:
            break

b1_shoulderpress_1.mp4


In [8]:
landmarks_df = pd.DataFrame(all_landmarks)
landmarks_df
#landmarks_df.to_csv("overhead.csv", index=False)

,label,x0,y0,z0,x1,y1,z1,x2,y2,z2,...,z29,x30,y30,z30,x31,y31,z31,x32,y32,z32
0,good,0.160414,0.336126,0.106198,0.150395,0.322298,0.042854,0.143350,0.320621,0.042659,...,0.476517,0.034147,0.957590,-0.339533,0.200701,0.915888,0.461650,0.105228,0.984931,-0.442127
1,good,0.191623,0.337395,0.413748,0.181381,0.324211,0.361956,0.175660,0.322811,0.361926,...,0.104903,0.276708,0.967693,-0.096427,0.210221,0.932068,0.064620,0.327011,0.934723,-0.180139
2,good,0.248386,0.347163,0.574578,0.240539,0.337967,0.497524,0.233260,0.338057,0.497610,...,0.001821,0.271848,0.956808,-0.051830,0.166852,0.950077,-0.037014,0.272785,0.947178,-0.112103
3,good,0.290136,0.350631,0.317024,0.279842,0.338213,0.243598,0.273157,0.337131,0.243568,...,0.209979,0.294017,0.979269,0.023859,0.171485,0.932095,0.151369,0.322534,0.946116,-0.070519


# ------------------------------------------------Bicep Curl------------------------------------------------

In [28]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
exit = False
folder_name="bicep"
path=f"{pt}/{folder_name}"


all_videos = []

for file in os.listdir(path):
    
    all_videos.append(os.fsdecode(file))

In [29]:
all_landmarks = []
for filename in all_videos:
    print(filename)
    cap = cv2.VideoCapture(folder_name + '/' + filename)
    with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.5) as pose:

        while cap.isOpened():
            ret, image = cap.read()

            if ret == False:
                break
            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            cv2.rectangle(image, (0,0), (225, 100), (0,0,0), -1)

            k = cv2.waitKey(1)
            
            if results.pose_landmarks is not None:      
                
                if k == 103:  #ASCII of g
                    label_landmarks(results.pose_landmarks, 'good')
                    cv2.putText(image, 'good', (60,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
               
                if k == 115:  #ASCII of s
                    label_landmarks(results.pose_landmarks, 'shoulder')
                    cv2.putText(image, 'shoulder', (60,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
          
                if k == 101:  #ASCII of e
                    label_landmarks(results.pose_landmarks, 'elbow')
                    cv2.putText(image, 'elbow', (60,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
          
                
            # Press ESC to quit
            if k == 27:
                exit = True
                break

            cv2.namedWindow('output', cv2.WINDOW_NORMAL) # Allows you to manually change the size of the window
            cv2.imshow('output', image)

        cap.release()
        cv2.destroyAllWindows()
        if exit == True:
            break

0.mp4
001 (1).mp4


In [30]:
landmarks_df = pd.DataFrame(all_landmarks)
landmarks_df
#landmarks_df.to_csv("bicep.csv", index=False)

,label,x0,y0,z0,x1,y1,z1,x2,y2,z2,...,z29,x30,y30,z30,x31,y31,z31,x32,y32,z32
0,good,0.494286,0.239073,-0.157815,0.501351,0.212765,-0.171348,0.505578,0.211542,-0.171418,...,0.114260,0.517839,1.093261,0.312962,0.537844,1.151748,-0.012034,0.473098,1.131454,0.218740
1,good,0.494460,0.240010,-0.154353,0.501551,0.213849,-0.169848,0.505754,0.212638,-0.169929,...,0.154148,0.516862,1.107005,0.287211,0.538186,1.150132,0.043776,0.472644,1.145627,0.187786
2,good,0.496649,0.240341,-0.189611,0.504322,0.214052,-0.200447,0.508813,0.212762,-0.200550,...,0.191025,0.516382,1.110553,0.316989,0.545839,1.157230,0.077398,0.476163,1.154651,0.210871
3,good,0.500003,0.238672,-0.198381,0.508779,0.211167,-0.207996,0.513641,0.209776,-0.208096,...,0.191308,0.515180,1.123386,0.318007,0.547438,1.170959,0.076103,0.476440,1.168173,0.208782
4,good,0.502506,0.234964,-0.216178,0.511144,0.207523,-0.222420,0.516205,0.206086,-0.222537,...,0.216337,0.514823,1.127965,0.310905,0.546537,1.172964,0.104035,0.476286,1.173575,0.197135
5,good,0.504117,0.227138,-0.219203,0.512766,0.200422,-0.223889,0.518037,0.198945,-0.224045,...,0.225515,0.513357,1.133150,0.309230,0.546465,1.177602,0.115595,0.476597,1.181115,0.197632
6,good,0.504565,0.224340,-0.214195,0.513277,0.198307,-0.220373,0.518519,0.196757,-0.220509,...,0.247055,0.512133,1.132863,0.316548,0.547856,1.176486,0.143200,0.477041,1.182878,0.207412
7,good,0.501644,0.210909,-0.280212,0.510233,0.184782,-0.284670,0.515039,0.182971,-0.284809,...,0.229440,0.491581,1.165751,0.294625,0.533947,1.222106,0.108927,0.455617,1.225081,0.172821
8,good,0.501208,0.209947,-0.269907,0.509967,0.183321,-0.274341,0.514811,0.181619,-0.274488,...,0.238191,0.490391,1.168621,0.316047,0.532806,1.224181,0.132830,0.454307,1.226956,0.205278
9,good,0.501022,0.209916,-0.270379,0.509735,0.183106,-0.275423,0.514615,0.181302,-0.275571,...,0.220205,0.490934,1.171516,0.345441,0.529281,1.257858,0.101173,0.454564,1.231838,0.239336
